# Traffic file processing

In [2]:
import csv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

os.chdir("/Users/Tom/Documents/EventsxTraffic/Traffic_Data")

### Remove empty rows

In [7]:
input_file = 'signal_id_mercedes_data.csv'
output_file = 'temporary_file.csv'

with open(input_file, 'r', newline='', encoding='utf-8') as csv_in, \
     open(output_file, 'w', newline='', encoding='utf-8') as csv_out:
    reader = csv.reader(csv_in)
    writer = csv.writer(csv_out)
    
    # Iterate over each row in the input CSV
    for row in reader:
        # Check if the row is not empty (i.e., at least one cell has content)
        if any(cell.strip() for cell in row):
            writer.writerow(row)

os.replace(output_file, input_file)


### Join long lat coordinates to approach volume

In [18]:
input_file = pd.read_csv('signal_id_mercedes_data.csv')
atlanta_signals_long_lat_df = pd.read_csv('atlanta_signals_long_lat.csv')

# Define a function to merge closest signal coordinates
def merge_coordinates(df, signal_column):
    return df.merge(
        atlanta_signals_long_lat_df, 
        left_on=signal_column, 
        right_on="SignalID", 
        suffixes=('', f'_{signal_column}_Coords')
    ).drop(columns=["SignalID"])

# Start with the closest_traffic_lights dataset and add coordinates for each closest signal
merged_df = input_file.copy()
merged_df = merge_coordinates(merged_df, f'Signal_ID')
merged_df = merged_df.rename(columns={'Signal_ID': f'SignalID'})

merged_df.to_csv("signal_id_data_lat_long.csv", index=False)


In [23]:
os.chdir("/Users/Tom/Documents/EventsxTraffic")

event_data = pd.read_csv('Events-Export-Atlanta-from-20240727-to-20240930.csv')

event_data = event_data[['title', 'category', 'start_local', 'end_local', 'phq_attendance', 'venue_name', 'lat', 'lon']]

event_data.head()

,title,category,start_local,end_local,phq_attendance,venue_name,lat,lon
0,Annual America's SBDC Conference,conferences,2024-09-09 09:00:00,2024-09-13 18:00:00,500.0,Atlanta Marriott Marquis,33.761585,-84.385615
1,IWF - International Woodworking Machinery and ...,expos,2024-08-06 00:00:00,2024-08-09 23:59:59,25524.0,Georgia World Congress Center,33.759006,-84.398739
2,International Woodworking Fair Atlanta,expos,2024-08-20 00:00:00,2024-08-23 23:59:59,20000.0,Georgia World Congress Center,33.759006,-84.398739
3,Annual International Conference,conferences,2024-09-24 09:00:00,2024-09-26 18:00:00,500.0,Hyatt Regency Atlanta,33.761681,-84.387093
4,Funny Girl,performing-arts,2024-08-01 19:30:00,2024-08-01 19:30:00,2950.0,The Fox Theatre,33.772585,-84.385603


In [26]:
df = merged_df

# Get distinct values in 'column1'
distinct_values = df['SignalID'].unique()

# Convert it to a DataFrame if you want a similar structure as SQL
distinct_df = pd.DataFrame(distinct_values, columns=['Signal_ID_distinct'])

print(distinct_df)

    Signal_ID_distinct
0               7186.0
1               7160.0
2               8145.0
3               7158.0
4               7181.0
5               8143.0
6               7177.0
7               7546.0
8                 60.0
9                 56.0
10                92.0
11                 8.0
12                29.0
13                46.0
14                 3.0
15                11.0
16              8142.0
17              8148.0
18              8136.0
19               115.0
20               132.0
21               165.0


In [10]:
# traffic_feature_engineering.py

# -------------------------------
# Step 1: Load and Preprocess Event Data
# -------------------------------

# Load the event data
event_data = pd.read_csv('Events-Export-Atlanta-from-20240727-to-20240930_copy.csv')

# Keep only the required columns
event_data = event_data[['title', 'category', 'start_local', 'end_local', 'phq_attendance', 'venue_name', 'lat', 'lon']]

# Convert time columns to datetime
event_data['start_time'] = pd.to_datetime(event_data['start_local'])
event_data['end_time'] = pd.to_datetime(event_data['end_local'])

# Drop the original start_local and end_local columns
event_data.drop(['start_local', 'end_local'], axis=1, inplace=True)

# Rename columns for consistency
event_data.rename(columns={
    'title': 'event_name',
    'category': 'event_category',
    'phq_attendance': 'event_weight',  # Assuming 'phq_attendance' is used as event weight
    'lat': 'event_latitude',
    'lon': 'event_longitude'
}, inplace=True)

# Handle missing values
event_data['event_weight'] = pd.to_numeric(event_data['event_weight'], errors='coerce').fillna(0)
event_data.dropna(subset=['event_latitude', 'event_longitude'], inplace=True)

# Compute event duration in hours
event_data['event_duration'] = (event_data['end_time'] - event_data['start_time']).dt.total_seconds() / 3600.0

# Optionally encode event categories using one-hot encoding
event_category_dummies = pd.get_dummies(event_data['event_category'], prefix='event_category')
event_data = pd.concat([event_data, event_category_dummies], axis=1)
event_data.drop('event_category', axis=1, inplace=True)

# -------------------------------
# Step 2: Load and Preprocess Traffic Signal Data
# -------------------------------

# Load the traffic data with the new schema
traffic_data = pd.read_csv('signal_id_mercedes_data.csv')

# Rename columns to match previous naming for consistency
traffic_data.rename(columns={
    'Signal_ID': 'signal_id',
    'StartDate': 'start_time',
    'EndDate': 'end_time',
    'WestboundVolume': 'volume_west',
    'EastboundVolume': 'volume_east',
    'NorthboundVolume': 'volume_north',
    'SouthboundVolume': 'volume_south'
}, inplace=True)

# Convert 'start_time' and 'end_time' to datetime with the corrected format
traffic_data['start_time'] = pd.to_datetime(traffic_data['start_time'], format='%m/%d/%y %H:%M')
traffic_data['end_time'] = pd.to_datetime(traffic_data['end_time'], format='%m/%d/%y %H:%M')

# Use 'start_time' as the 'timestamp' for consistency
traffic_data['timestamp'] = traffic_data['start_time']

# Drop 'start_time' and 'end_time' if not needed
# traffic_data.drop(['start_time', 'end_time'], axis=1, inplace=True)

# Load signal locations (if available)
signal_locations = pd.read_csv('atlanta_signals_long_lat.csv')

# Merge traffic data with signal locations
traffic_data = traffic_data.merge(signal_locations, on='signal_id', how='left')

# Ensure latitude and longitude are numeric
traffic_data['latitude'] = pd.to_numeric(traffic_data['latitude'], errors='coerce')
traffic_data['longitude'] = pd.to_numeric(traffic_data['longitude'], errors='coerce')

# Handle missing values in location data
traffic_data.dropna(subset=['latitude', 'longitude'], inplace=True)

# Compute total traffic volume
traffic_data['total_volume'] = traffic_data[['volume_north', 'volume_south', 'volume_east', 'volume_west']].sum(axis=1)

# -------------------------------
# Step 3: Feature Engineering
# -------------------------------

# Sort traffic data by signal_id and timestamp
traffic_data.sort_values(by=['signal_id', 'timestamp'], inplace=True)

# Reset index after sorting
traffic_data.reset_index(drop=True, inplace=True)

# Create lag features for previous total volumes
for lag in range(1, 4):
    traffic_data[f'total_volume_lag_{lag}'] = traffic_data.groupby('signal_id')['total_volume'].shift(lag)

# Fill missing values in lag features with zeros
lag_features = [f'total_volume_lag_{lag}' for lag in range(1, 4)]
traffic_data[lag_features] = traffic_data[lag_features].fillna(0)

# Extract time-based features
traffic_data['hour'] = traffic_data['timestamp'].dt.hour
traffic_data['day_of_week'] = traffic_data['timestamp'].dt.dayofweek  # Monday=0, Sunday=6
traffic_data['is_weekend'] = traffic_data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

# Encode 'signal_id'
le_signal_id = LabelEncoder()
traffic_data['signal_id_encoded'] = le_signal_id.fit_transform(traffic_data['signal_id'])

# Initialize columns for event features
event_feature_cols = [
    'num_events',
    'total_event_weight',
    'avg_distance_to_events',
    'min_distance_to_event',
    'avg_event_duration',
    'avg_time_since_event_start',
    'avg_time_until_event_end'
]

# Include event category dummy columns if using one-hot encoding
event_category_cols = event_category_dummies.columns.tolist()
event_feature_cols.extend(event_category_cols)

for col in event_feature_cols:
    traffic_data[col] = 0

# Function to compute event features for each traffic record
def get_nearby_event_features(row):
    timestamp = row['timestamp']
    signal_coords = (row['latitude'], row['longitude'])
    
    # Find active events at the timestamp
    active_events = event_data[
        (event_data['start_time'] <= timestamp) & (event_data['end_time'] >= timestamp)
    ].copy()
    
    if active_events.empty:
        # No active events
        return pd.Series({
            'num_events': 0,
            'total_event_weight': 0,
            'avg_distance_to_events': np.nan,
            'min_distance_to_event': np.nan,
            'avg_event_duration': 0,
            'avg_time_since_event_start': 0,
            'avg_time_until_event_end': 0,
            **{col: 0 for col in event_category_cols}
        })
    else:
        # Calculate distances to events
        active_events['distance'] = active_events.apply(
            lambda x: geodesic(signal_coords, (x['event_latitude'], x['event_longitude'])).kilometers, axis=1)
        
        # Filter events within a certain radius (e.g., 1 km)
        nearby_events = active_events[active_events['distance'] <= 1.0]
        
        if nearby_events.empty:
            # No nearby events
            return pd.Series({
                'num_events': 0,
                'total_event_weight': 0,
                'avg_distance_to_events': np.nan,
                'min_distance_to_event': np.nan,
                'avg_event_duration': 0,
                'avg_time_since_event_start': 0,
                'avg_time_until_event_end': 0,
                **{col: 0 for col in event_category_cols}
            })
        else:
            # Compute aggregate features
            num_events = len(nearby_events)
            total_event_weight = nearby_events['event_weight'].sum()
            avg_distance = nearby_events['distance'].mean()
            min_distance = nearby_events['distance'].min()
            avg_event_duration = nearby_events['event_duration'].mean()
            nearby_events['time_since_event_start'] = (timestamp - nearby_events['start_time']).dt.total_seconds() / 3600.0
            nearby_events['time_until_event_end'] = (nearby_events['end_time'] - timestamp).dt.total_seconds() / 3600.0
            avg_time_since_start = nearby_events['time_since_event_start'].mean()
            avg_time_until_end = nearby_events['time_until_event_end'].mean()
            
            # Aggregate event category features
            category_sums = nearby_events[event_category_cols].sum()
            category_features = category_sums.to_dict()
            
            # Combine all features into a single series
            features = {
                'num_events': num_events,
                'total_event_weight': total_event_weight,
                'avg_distance_to_events': avg_distance,
                'min_distance_to_event': min_distance,
                'avg_event_duration': avg_event_duration,
                'avg_time_since_event_start': avg_time_since_start,
                'avg_time_until_event_end': avg_time_until_end
            }
            # Add category features
            features.update(category_features)
            
            return pd.Series(features)

# Apply the function to the traffic data
event_features = traffic_data.apply(get_nearby_event_features, axis=1)

# Merge the event features back into traffic_data
traffic_data.update(event_features)

# Handle missing values in distance features
max_distance = traffic_data[['avg_distance_to_events', 'min_distance_to_event']].max().max()
traffic_data['avg_distance_to_events'].fillna(max_distance, inplace=True)
traffic_data['min_distance_to_event'].fillna(max_distance, inplace=True)

# Replace any remaining NaNs with zeros
traffic_data.fillna(0, inplace=True)

# Create interaction features
traffic_data['num_events_hour'] = traffic_data['num_events'] * traffic_data['hour']
traffic_data['total_event_weight_day_of_week'] = traffic_data['total_event_weight'] * traffic_data['day_of_week']

# -------------------------------
# Step 4: Finalize Feature Set
# -------------------------------

# Define the final list of feature columns
feature_cols = [
    'num_events',
    'total_event_weight',
    'avg_distance_to_events',
    'min_distance_to_event',
    'avg_event_duration',
    'avg_time_since_event_start',
    'avg_time_until_event_end',
    'hour',
    'day_of_week',
    'is_weekend',
    'signal_id_encoded',
    # Lag features
    'total_volume_lag_1',
    'total_volume_lag_2',
    'total_volume_lag_3',
    # Interaction features
    'num_events_hour',
    'total_event_weight_day_of_week'
] + event_category_cols  # Include event category features if used

# Select the final dataset for modeling
model_data = traffic_data[feature_cols + ['total_volume']]

# -------------------------------
# Step 5: Save the Prepared Data
# -------------------------------

# Save the prepared data to a CSV file
model_data.to_csv('traffic_data_prepared.csv', index=False)

# -------------------------------
# Script Completed
# -------------------------------

print("Feature engineering completed. Prepared data saved to 'traffic_data_prepared.csv'.")

Feature engineering completed. Prepared data saved to 'traffic_data_prepared.csv'.


/var/folders/_k/658z_90d41z8_024w56tsnqh0000gq/T/ipykernel_39901/2823516375.py:207: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.         0.         0.         ... 0.99223203 0.99223203 0.        ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  traffic_data.update(event_features)
/var/folders/_k/658z_90d41z8_024w56tsnqh0000gq/T/ipykernel_39901/2823516375.py:207: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.         0.         0.         ... 0.99223203 0.99223203 0.        ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  traffic_data.update(event_features)
/var/folders/_k/658z_90d41z8_024w56tsnqh0000gq/T/ipykernel_39901/2823516375.py:207: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value